## 算法介绍

  
  DenseNet是一种具有密集连接的卷积神经网络。在该网络中，任何两层之间都有直接的连接，也就是说，网络每一层的输入都是前面所有层输出的并集，而该层所学习的特征图也会被直接传给其后面所有层作为输入。下图为网络中一个DenseBlock的结构，每一层都与前面所有层在channel维度上进行concat，作为下一层输入，这样对于一个L层的网络，DenseNet共包含![CodeCogsEqn1](CodeCogsEqn1.png)个连接。
  
  ![fig1](fig1.png)
  
  用公式表示传统神经网络、ResNet和DenseNet结构的不同：  
  （1）传统神经网络l层的输出为： 
  
  ![CodeCogEqn2](CodeCogsEqn2.png)  
  
  (2) 对于ResNet，其l层的输出为：
  
  ![CodeCogEqn3](CodeCogsEqn3.png)  
  
  (3) 对于DenseNet，其l层的输出为：
  
  ![CodeCogEqn4](CodeCogsEqn4.png)
  
  DenseNet网络结构优点如下：
  - 相比传统网络，所需参数更少
  - 由于每一层都可以获取到原始输入信息，利于整个网络的梯度传播
  - 在相对较少的训练数据情况下，能有效防止过拟合
  
  下图是一个包含三个DenseBlock的整体网络结构：   
  
  ![fig2](fig2.png)
  
  每个DenseBlock中的feature map 尺寸要一致，DenseBlock之间通过Transition模块进行连接，下面对DenseBlock以及Transition的结构以及编码实现进行说明。 

## 算法实现

DenseBlock中各个层的特征图大小一致，可以在channel维度上连接。DenseBlock中的非线性组合函数 H(.) 采用的是BN+ReLU+3x3 Conv的结构。所有DenseBlock中各个层卷积之后均输出 k 个特征图，即得到的特征图的channel数为 k ，或者说采用 k 个卷积核。 k 为growth rate，这是一个超参数，一般情况下使用较小的 k （比如12），就可以得到较佳的性能。假定输入层的特征图的channel数为 ![CodeCogEqn5](CodeCogsEqn5.png) ，那么 l 层输入的channel数为 ![CodeCogsEqn6](CodeCogsEqn6.png)，因此随着层数增加，尽管 k 设定得较小，DenseBlock的输入会非常多，不过这是由于特征重用所造成的，每个层仅有 k 个特征是自己独有的。由于后面层的输入会非常多，DenseBlock内部可以采用bottleneck层来减少计算量，主要是原有的结构中增加1x1 Conv，即BN+ReLU+1x1 Conv+BN+ReLU+3x3 Conv，该结构称为DenseNet-B结构。其中1x1 Conv得到 4k 个特征图，其作用是降低特征数量，从而提升计算效率。结构图如下： 
  
  ![fig3](fig3.png)
  
  基于结构图，其单个conv block编码实现为：

In [1]:
import keras

def conv_block(x, growth_rate, name):
    '''
    conv block
    :param x: input tensor
    :param growth_rate: integer, output feature map channel
    :param name: string, dense block name
    
    returns:
    output tensor
    '''
    x1 = keras.layers.normalization.BatchNormalization(axis = 3, name = name + '_0_bn')(x)
    x1 = keras.layers.Activation('relu', name = name + '_0_relu')(x1)
    x1 = keras.layers.convolutional.Conv2D(filters = 4 * growth_rate, kernel_size = (1, 1), strides = (1, 1), padding = 'same',
                                         kernel_initializer = 'glorot_uniform', use_bias = False, name = name + '_0_conv')(x1)
    
    x1 = keras.layers.normalization.BatchNormalization(axis = 3, name = name + '_1_bn')(x1)
    x1 = keras.layers.Activation('relu', name = name + '_1_relu')(x1)
    x1 = keras.layers.convolutional.Conv2D(filters = growth_rate, kernel_size = (3, 3), strides = (1, 1), padding = 'same',
                                          kernel_initializer = 'glorot_uniform', use_bias = False, name = name + '_1_conv')(x1)
    
    x = keras.layers.Concatenate(axis = 3, name = name + '_concat')([x, x1])#保留每层得到的feature map
    return x

/usr/local/SkyCompute/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


DenseBlock是由多个conv block组成的，根据conv block，其DenseBlock编码实现为：

In [2]:
#不同数量的conv_block组成了DenseBlock模块
def dense_block(x, blocks, name):
    '''
    dense block
    :param x: input tensor
    :param blocks: integer, number of bottleneck layers
    :param name: string , name of dense block
    
    returns:
    output tensor    
    '''
    for i in range(blocks):
        x = conv_block(x, 32, name = name + '_convblock'+str(i+1))
    
    return x

Transition层主要用于连接两个相邻的DenseBlock，并且降低特征图大小，结构为 BN+ReLU+1x1 Conv+2x2 AvgPooling。另外，Transition层可以起到压缩模型的作用，如果一个DenseBlock输出 m 个feature map，Transition层可以生成 ![CodeCogsEqn7](CodeCogsEqn7.png)个feature map，其中，![CodeCogsEqn8](CodeCogsEqn8.png)是压缩系数，当![CodeCogsEqn8](CodeCogsEqn8.png)=1时，特征个数经过Transiton层未发生变化，无压缩；当![CodeCogsEqn8](CodeCogsEqn8.png)<1时，结构称为DenseNet-C。下面是Transition的编码实现，其中，![CodeCogsEqn8](CodeCogsEqn8.png)设置为0.5。

In [3]:
def transition_block(x, theta, name):
    '''
    trainsition block
    :param x: input tensor.
    :param theta: float, compression rate at transition layers.
    :param name: string, block label.
    
    returns:
    output tensor
    '''
    x = keras.layers.normalization.BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_bn')(x)#epsilon防止除零错误
    x = keras.layers.Activation('relu', name=name + '_relu')(x)
    x = keras.layers.convolutional.Conv2D(filters = int(keras.backend.int_shape(x)[3] * theta), kernel_size = (1, 1), strides = (1, 1), padding = 'same',
                                          kernel_initializer='glorot_uniform', use_bias=False, name=name + '_conv')(x)
    x = keras.layers.pooling.AveragePooling2D(pool_size = (2, 2), strides=2, name=name + '_pool')(x)
    return x

  ConvBlock和Transition模块实现完毕后，我们可以很容易的对DenseNet网络结构进行搭建，其不同层次的结构组成如下表：

  ![fig4](fig4.png)
  
  我们以DenseNet-121为例进行编码实现：

In [4]:
def DenseNet121(input_shape=(224,224,3), classes = 1000):
    '''
    DenseNet121
    :param input_shape:  tuple, input tensor shape
    :param classes: integer, classes defined by your dataset
    
    returns:   
    keras model
    '''
    x_input = keras.layers.Input(input_shape)
    
    x = keras.layers.convolutional.Conv2D(64, 7, strides=2, padding='same', use_bias=False, name='conv1/conv')(x_input)
    x = keras.layers.normalization.BatchNormalization(axis=3, epsilon=1.001e-5,name='conv1/bn')(x)
    x = keras.layers.Activation('relu', name='conv1/relu')(x)
    x = keras.layers.pooling.MaxPooling2D(3, strides=2, padding='same', name='pool1')(x)
    
    x = dense_block(x, 6, name='dense_block1')
    x = transition_block(x, 0.5, name = 'transition_block_1')
    
    x = dense_block(x, 12, name = 'dense_block_2')
    x = transition_block(x, 0.5, name = 'transition_block_2')
    
    x = dense_block(x, 24, name = 'dense_block_3')
    x = transition_block(x, 0.5, name = 'transition_block_3')
    
    x = dense_block(x, 16, name = 'dense_block_4')
    x = keras.layers.normalization.BatchNormalization(axis=3, epsilon=1.001e-5, name='bn')(x)
    
    #classification
    x = keras.layers.pooling.GlobalAveragePooling2D(name = 'global_avg_pooling')(x)
    x = keras.layers.core.Dense(classes, activation='softmax', name = 'classification')(x)
    
    model = keras.models.Model(inputs = x_input, outputs = x, name = 'DenseNet121')
    return model

In [5]:
model = DenseNet121()
print(model.summary())

(?, 1024)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu (Activation)         (None, 112, 112, 64) 0           conv1/bn[0][0]                   
__________________________________________________________________________________________________


## 参考文献
(1) Huang G, Liu Z, Van Der Maaten L, et al. Densely connected convolutional networks[C]//CVPR. 2017, 1(2): 3.  
(2) https://zhuanlan.zhihu.com/p/37189203